Notebook:

Creates the wage outcome data. We only have aggregate wage for powiat. 

Logic: Provide wage, and employment (denominator of average wage), such they can be aggregated across powiats into our clusters

In [12]:
import sys
from pathlib import Path

p = Path.cwd().resolve()
repo_root = next((parent for parent in [p] + list(p.parents) if (parent / ".git").exists()), None)
if repo_root is None:
    raise RuntimeError("Repo root not found. Open the repo folder in VS Code.")

sys.path.insert(0, str(repo_root))
print("Repo root:", repo_root)

import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt

Repo root: C:\Users\harri\OneDrive - Imperial College London\Year 3 Group Project\Group_Project_Y3


**Read in data**

In [13]:
ptot = pd.read_csv(repo_root / "cleaned/03_01_outcome_tables/population_powiat.csv", index_col=0)
ptot

,county_code,year,county_kts,county_name,tp1_nc_pop,tp2_yr_pop,tp3b_nc_active,tp3a_nc_employed,tp4a_employed,tp4b_active,tp5a_employed_18-64,tp5c_employed,tp5b_active,tp6a_employed_18-64,tp6c_employed,tp6b_active
0,201.0,1995,10030210101000,Powiat bolesławiecki,76739,63763.0,40280.0,38792.0,35835.0,37238.0,NaN,NaN,NaN,NaN,NaN,NaN
1,201.0,1996,10030210101000,Powiat bolesławiecki,76739,64107.0,40280.0,38792.0,36028.0,37438.0,NaN,NaN,NaN,NaN,NaN,NaN
2,201.0,1997,10030210101000,Powiat bolesławiecki,76739,64719.0,40280.0,38792.0,36372.0,37796.0,NaN,NaN,NaN,NaN,NaN,NaN
3,201.0,1998,10030210101000,Powiat bolesławiecki,76739,65261.0,40280.0,38792.0,36677.0,38112.0,NaN,NaN,NaN,NaN,NaN,NaN
4,201.0,1999,10030210101000,Powiat bolesławiecki,76739,63818.0,40280.0,38792.0,35866.0,37270.0,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11395,3263.0,2020,10023216663000,Powiat m. Świnoujście,35988,35382.0,17737.0,16905.0,19036.0,19991.0,18426.0,18823.0,19460.0,13288.0,18754.0,19634.0
11396,3263.0,2021,10023216663000,Powiat m. Świnoujście,35988,34960.0,17737.0,16905.0,18808.0,19752.0,18423.0,18878.0,19578.0,13286.0,18808.0,19752.0
11397,3263.0,2022,10023216663000,Powiat m. Świnoujście,35988,34619.0,17737.0,16905.0,18625.0,19560.0,18431.0,19214.0,19664.0,13292.0,19142.0,19839.0
11398,3263.0,2023,10023216663000,Powiat m. Świnoujście,35988,34317.0,17737.0,16905.0,18463.0,19389.0,18407.0,19218.0,19664.0,13275.0,19146.0,19839.0


In [14]:
p2497 = pd.read_csv(repo_root / "cleaned/03_01_outcome_data/wage_powiat_p2497.csv", index_col=0)
p2497

,code,powiat,type,year,value
0,201000,Powiat bolesławiecki,grand total,2002,1873.59
1,202000,Powiat dzierżoniowski,grand total,2002,1703.68
2,203000,Powiat głogowski,grand total,2002,1868.60
3,204000,Powiat górowski,grand total,2002,1730.53
4,205000,Powiat jaworski,grand total,2002,1705.19
...,...,...,...,...,...
9139,3217000,Powiat wałecki,grand total,2025,NaN
9140,3218000,Powiat łobeski,grand total,2025,NaN
9141,3261000,City with powiat status Koszalin,grand total,2025,NaN
9142,3262000,City with powiat status Szczecin,grand total,2025,NaN


In [15]:
p2497.code.value_counts()

code
201000     24
202000     24
203000     24
204000     24
205000     24
           ..
3217000    24
3218000    24
3261000    24
3262000    24
3263000    24
Name: count, Length: 381, dtype: int64

In [16]:
# What is the extra county code? - use county codes to tell
county_codes = pd.read_csv(repo_root / "cleaned/00_codes/county_codes.csv")

p2497["merge_code"] = p2497["code"].apply(lambda x: int(str(x)[:-3]))

p2497 = p2497.merge(
    county_codes,
    how="left",
    left_on="merge_code",
    right_on="county_code"
)

p2497[p2497["county_code"].isna()]

,code,powiat,type,year,value,merge_code,county_code,county_kts,county_name
28,263000,City with powiat status Wałbrzych to 2002,grand total,2002,2117.52,263,NaN,NaN,NaN
409,263000,City with powiat status Wałbrzych to 2002,grand total,2003,NaN,263,NaN,NaN,NaN
790,263000,City with powiat status Wałbrzych to 2002,grand total,2004,NaN,263,NaN,NaN,NaN
1171,263000,City with powiat status Wałbrzych to 2002,grand total,2005,NaN,263,NaN,NaN,NaN
1552,263000,City with powiat status Wałbrzych to 2002,grand total,2006,NaN,263,NaN,NaN,NaN
1933,263000,City with powiat status Wałbrzych to 2002,grand total,2007,NaN,263,NaN,NaN,NaN
2314,263000,City with powiat status Wałbrzych to 2002,grand total,2008,NaN,263,NaN,NaN,NaN
2695,263000,City with powiat status Wałbrzych to 2002,grand total,2009,NaN,263,NaN,NaN,NaN
3076,263000,City with powiat status Wałbrzych to 2002,grand total,2010,NaN,263,NaN,NaN,NaN
3457,263000,City with powiat status Wałbrzych to 2002,grand total,2011,NaN,263,NaN,NaN,NaN


In [17]:
p2497[p2497["powiat"].apply(lambda x: "Wałbrzy" in x)]

,code,powiat,type,year,value,merge_code,county_code,county_kts,county_name
28,263000,City with powiat status Wałbrzych to 2002,grand total,2002,2117.52,263,NaN,NaN,NaN
30,265000,City with powiat status Wałbrzych since 2013,grand total,2002,NaN,265,265.0,1.003021e+13,Powiat m. Wałbrzych
409,263000,City with powiat status Wałbrzych to 2002,grand total,2003,NaN,263,NaN,NaN,NaN
411,265000,City with powiat status Wałbrzych since 2013,grand total,2003,NaN,265,265.0,1.003021e+13,Powiat m. Wałbrzych
790,263000,City with powiat status Wałbrzych to 2002,grand total,2004,NaN,263,NaN,NaN,NaN
792,265000,City with powiat status Wałbrzych since 2013,grand total,2004,NaN,265,265.0,1.003021e+13,Powiat m. Wałbrzych
1171,263000,City with powiat status Wałbrzych to 2002,grand total,2005,NaN,263,NaN,NaN,NaN
1173,265000,City with powiat status Wałbrzych since 2013,grand total,2005,NaN,265,265.0,1.003021e+13,Powiat m. Wałbrzych
1552,263000,City with powiat status Wałbrzych to 2002,grand total,2006,NaN,263,NaN,NaN,NaN
1554,265000,City with powiat status Wałbrzych since 2013,grand total,2006,NaN,265,265.0,1.003021e+13,Powiat m. Wałbrzych


In [18]:
missing = p2497["county_code"].isna()
p2497.loc[missing, "county_code"] = 265
p2497.loc[missing, "county_kts"] = 10030210365000
p2497.loc[missing, "county_name"] = "Powiat m. Wałbrzych"

In [19]:
p2497.dropna(subset=["value"], inplace=True)
p2497

,code,powiat,type,year,value,merge_code,county_code,county_kts,county_name
0,201000,Powiat bolesławiecki,grand total,2002,1873.59,201,201.0,1.003021e+13,Powiat bolesławiecki
1,202000,Powiat dzierżoniowski,grand total,2002,1703.68,202,202.0,1.003021e+13,Powiat dzierżoniowski
2,203000,Powiat głogowski,grand total,2002,1868.60,203,203.0,1.003021e+13,Powiat głogowski
3,204000,Powiat górowski,grand total,2002,1730.53,204,204.0,1.003021e+13,Powiat górowski
4,205000,Powiat jaworski,grand total,2002,1705.19,205,205.0,1.003021e+13,Powiat jaworski
...,...,...,...,...,...,...,...,...,...
8758,3217000,Powiat wałecki,grand total,2024,7568.91,3217,3217.0,1.002322e+13,Powiat wałecki
8759,3218000,Powiat łobeski,grand total,2024,7375.31,3218,3218.0,1.002322e+13,Powiat łobeski
8760,3261000,City with powiat status Koszalin,grand total,2024,7872.25,3261,3261.0,1.002322e+13,Powiat m. Koszalin
8761,3262000,City with powiat status Szczecin,grand total,2024,9044.66,3262,3262.0,1.002322e+13,Powiat m. Szczecin


In [20]:
p2497.county_code.value_counts()

county_code
3263.0    23
201.0     23
202.0     23
203.0     23
204.0     23
          ..
216.0     23
215.0     23
214.0     23
213.0     23
265.0     13
Name: count, Length: 380, dtype: int64

**Merge datasets:**

In [21]:
# Set up p2497 for merging
p2497 = p2497[["county_code", "year", "value"]].rename(columns={"value":"wage"}).set_index(["county_code", "year"])

# Set up ptot for merging
ptot = ptot.set_index(["county_code", "year"])
# keep only relevant columns
ptot = ptot[[
    "county_kts", "county_name",
    "tp1_nc_pop", "tp2_yr_pop", "tp3a_nc_employed", "tp4a_employed", "tp5a_employed_18-64", "tp5c_employed", "tp6a_employed_18-64", "tp6c_employed"
]]

# Merge
wage = p2497.join(ptot).reset_index()

# Save
wage.to_csv(repo_root / "cleaned/03_01_outcome_tables/wage_table.csv")

wage

,county_code,year,wage,county_kts,county_name,tp1_nc_pop,tp2_yr_pop,tp3a_nc_employed,tp4a_employed,tp5a_employed_18-64,tp5c_employed,tp6a_employed_18-64,tp6c_employed
0,201.0,2002,1873.59,10030210101000,Powiat bolesławiecki,76739,71744.0,38792.0,40320.0,NaN,NaN,NaN,NaN
1,202.0,2002,1703.68,10030210302000,Powiat dzierżoniowski,86543,89409.0,39657.0,46850.0,NaN,NaN,NaN,NaN
2,203.0,2002,1868.60,10030210203000,Powiat głogowski,75191,72322.0,36209.0,38258.0,NaN,NaN,NaN,NaN
3,204.0,2002,1730.53,10030210204000,Powiat górowski,29042,29108.0,13765.0,15049.0,NaN,NaN,NaN,NaN
4,205.0,2002,1705.19,10030210105000,Powiat jaworski,42493,43339.0,20715.0,23403.0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
8725,3217.0,2024,7568.91,10023216417000,Powiat wałecki,44491,42709.0,21486.0,22849.0,24059.0,23703.0,17254.0,23484.0
8726,3218.0,2024,7375.31,10023216418000,Powiat łobeski,30007,28560.0,13463.0,14280.0,15803.0,15851.0,10592.0,14677.0
8727,3261.0,2024,7872.25,10023216361000,Powiat m. Koszalin,93083,91401.0,46160.0,50545.0,49106.0,50728.0,36402.0,51949.0
8728,3262.0,2024,9044.66,10023216562000,Powiat m. Szczecin,349790,337152.0,179608.0,193862.0,185569.0,187119.0,143032.0,199247.0
